# 🤖 Your First Intelligent Agent Communication Pair

**Welcome to the magic moment!** In this notebook, you'll build your first pair of intelligent agents that communicate using the A2A protocol and make decisions using OpenAI.

## What You'll Build in 20 Minutes

✅ **Intelligent A2A Agents** - Agents that think before responding using OpenAI  
✅ **Structured Message Protocol** - Professional message format with validation  
✅ **Real Business Scenario** - Customer inquiry routing system  
✅ **Error Handling** - Graceful failure management and retry logic  
✅ **Message Traceability** - Complete audit trail of agent communications  
✅ **Performance Monitoring** - Response times and success rates  

## Why This Matters

**This is the foundation** of all advanced A2A systems. Every enterprise agent network starts with two agents successfully communicating. You're about to see:

- **Agents that actually think** - Not just scripted responses, but intelligent decision-making
- **True collaboration** - Agents working together to solve problems
- **Business value** - Real applications you can deploy immediately
- **Professional implementation** - Enterprise-grade code you can extend

## The Agent Pair You'll Create

**🎯 CustomerServiceAgent** - Handles incoming customer inquiries with intelligence  
**🧠 TechnicalSupportAgent** - Provides specialized technical assistance  

**Ready to witness AI agents collaborating intelligently?** Let's build the future! 🚀

---

## 📋 Step 1: Import Dependencies & Verify Environment

First, let's make sure everything from Notebook 00 is working perfectly and import the libraries we'll need for intelligent agent communication.

In [ ]:
# Import all required libraries for A2A intelligent agents
import os
import json
import asyncio
import time
from datetime import datetime, timezone
from typing import Dict, List, Optional, Any
from dataclasses import dataclass, asdict
from enum import Enum

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# OpenAI for intelligent responses
import openai

# Data validation
from pydantic import BaseModel, Field, validator

# Logging
from loguru import logger

print("🔧 A2A Development Environment Check:")
print("=" * 40)

# Verify OpenAI setup
api_key = os.getenv('OPENAI_API_KEY')
if api_key and len(api_key) > 20:
    print(f"✅ OpenAI API Key: {api_key[:8]}...{api_key[-4:]}")
    client = openai.OpenAI(api_key=api_key)
    model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
    print(f"✅ Default Model: {model}")
else:
    print("❌ OpenAI API key not configured")
    print("   Please check your .env file")

# Configure logging for agent communication
logger.add("agent_communication.log", rotation="1 MB", level="INFO")
print("✅ Logging configured for agent monitoring")

print("✅ All dependencies loaded successfully")
print("🚀 Ready to build intelligent A2A agents!")

---

## 🏗️ Step 2: Define A2A Message Protocol

**Before agents can communicate**, we need a standardized message format. This is the foundation of the A2A protocol - structured, validated, and traceable messages.

### Why Message Structure Matters:
- **Reliability**: Consistent format ensures messages are always understood
- **Traceability**: Every message has a unique ID and timestamp
- **Validation**: Pydantic ensures message integrity
- **Extensibility**: Easy to add new message types and fields

In [ ]:
# Define message types for A2A communication
class MessageType(str, Enum):
    """Standardized message types for A2A protocol"""
    GREETING = "greeting"
    REQUEST = "request"
    RESPONSE = "response"
    TASK_ASSIGNMENT = "task_assignment"
    TASK_COMPLETION = "task_completion"
    ERROR = "error"
    HANDOFF = "handoff"
    STATUS_UPDATE = "status_update"

class MessagePriority(str, Enum):
    """Message priority levels"""
    LOW = "low"
    NORMAL = "normal"
    HIGH = "high"
    URGENT = "urgent"

class A2AMessage(BaseModel):
    """Standardized A2A message format with validation"""
    
    # Core message identification
    id: str = Field(..., description="Unique message identifier")
    conversation_id: str = Field(..., description="Conversation thread identifier")
    
    # Agent information
    from_agent: str = Field(..., description="Sending agent name")
    from_role: str = Field(..., description="Sending agent role")
    to_agent: str = Field(..., description="Receiving agent name")
    to_role: str = Field(..., description="Receiving agent role")
    
    # Message content
    message_type: MessageType = Field(..., description="Type of message")
    content: str = Field(..., description="Message content")
    priority: MessagePriority = Field(default=MessagePriority.NORMAL, description="Message priority")
    
    # Additional data (optional)
    metadata: Dict[str, Any] = Field(default_factory=dict, description="Additional message data")
    
    # Protocol information
    protocol_version: str = Field(default="A2A-v1.0", description="Protocol version")
    timestamp: str = Field(default_factory=lambda: datetime.now(timezone.utc).isoformat(), description="Message timestamp")
    
    # Response tracking
    requires_response: bool = Field(default=True, description="Whether this message requires a response")
    response_timeout: int = Field(default=30, description="Response timeout in seconds")
    
    @validator('id')
    def validate_id(cls, v):
        if not v or len(v) < 5:
            raise ValueError('Message ID must be at least 5 characters')
        return v

# Test the message protocol
print("🔧 Testing A2A Message Protocol:")
print("=" * 35)

# Create a test message
test_message = A2AMessage(
    id="msg_001_test",
    conversation_id="conv_001",
    from_agent="TestAgent",
    from_role="Tester",
    to_agent="TargetAgent",
    to_role="Receiver",
    message_type=MessageType.GREETING,
    content="Hello! Testing A2A protocol.",
    metadata={"test": True, "version": "1.0"}
)

print("✅ A2A Message Protocol validated successfully!")
print(f"📋 Message ID: {test_message.id}")
print(f"📋 Protocol Version: {test_message.protocol_version}")
print(f"📋 Message Type: {test_message.message_type}")
print(f"📋 Timestamp: {test_message.timestamp}")

# Show message as JSON (how it would be transmitted)
print("\n📤 Message JSON format:")
print(json.dumps(test_message.dict(), indent=2))

**Perfect!** Our A2A message protocol is working. Every message is:
- ✅ **Uniquely identified** with ID and conversation tracking
- ✅ **Fully traceable** with timestamps and agent information  
- ✅ **Validated** using Pydantic for data integrity
- ✅ **Extensible** with metadata for custom data

---

## 🧠 Step 3: Build the Intelligent Agent Base Class

Now we'll create the foundation for all intelligent A2A agents. This base class handles:
- **OpenAI integration** for intelligent responses
- **Message sending and receiving** with validation
- **Error handling** and retry logic
- **Performance monitoring** and logging

In [ ]:
class IntelligentA2AAgent:
    """Base class for intelligent A2A agents with OpenAI integration"""
    
    def __init__(self, name: str, role: str, specialization: str = ""):
        self.name = name
        self.role = role
        self.specialization = specialization
        self.message_count = 0
        self.conversation_history = []
        self.performance_metrics = {
            "messages_sent": 0,
            "messages_received": 0,
            "avg_response_time": 0,
            "successful_interactions": 0,
            "failed_interactions": 0
        }
        
        # OpenAI client setup
        self.openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        self.model = os.getenv('OPENAI_MODEL', 'gpt-4o-mini')
        
        logger.info(f"🤖 {self.name} ({self.role}) initialized with {self.specialization}")
        print(f"🤖 {self.name} ({self.role}) is online and ready!")
    
    def _generate_message_id(self) -> str:
        """Generate unique message ID"""
        self.message_count += 1
        timestamp = datetime.now().strftime("%H%M%S%f")[:8]
        return f"msg_{self.name}_{self.message_count}_{timestamp}"
    
    def _get_conversation_id(self, other_agent: str) -> str:
        """Generate conversation ID for agent pair"""
        agents = sorted([self.name, other_agent])
        return f"conv_{agents[0]}_{agents[1]}_{datetime.now().strftime('%Y%m%d')}"
    
    async def _get_intelligent_response(self, message_content: str, context: str = "") -> str:
        """Generate intelligent response using OpenAI"""
        try:
            # Build the prompt with agent context
            system_prompt = f"""
You are {self.name}, a {self.role} agent in an A2A (Agent-to-Agent) communication network.
Your specialization: {self.specialization}

Key instructions:
1. Respond professionally as this specific agent role
2. Be helpful, concise, and action-oriented
3. If you need to involve another agent, be specific about why
4. Always maintain the context of A2A protocol communication
5. Respond in a way that advances the conversation toward resolution

Context: {context}
"""
            
            start_time = time.time()
            
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": message_content}
                ],
                max_tokens=200,
                temperature=0.7
            )
            
            response_time = time.time() - start_time
            
            # Update performance metrics
            self._update_response_time(response_time)
            
            intelligent_response = response.choices[0].message.content
            
            logger.info(f"🧠 {self.name} generated intelligent response in {response_time:.2f}s")
            
            return intelligent_response
            
        except Exception as e:
            logger.error(f"❌ {self.name} failed to generate intelligent response: {str(e)}")
            # Fallback to basic response
            return f"I'm {self.name}, a {self.role}. I received your message but couldn't generate an optimal response right now. How can I assist you?"
    
    def _update_response_time(self, response_time: float):
        """Update average response time metrics"""
        current_avg = self.performance_metrics["avg_response_time"]
        message_count = self.performance_metrics["messages_sent"]
        
        if message_count == 0:
            self.performance_metrics["avg_response_time"] = response_time
        else:
            # Calculate rolling average
            self.performance_metrics["avg_response_time"] = (
                (current_avg * message_count + response_time) / (message_count + 1)
            )
    
    async def send_message(
        self, 
        to_agent: str, 
        to_role: str, 
        content: str, 
        message_type: MessageType = MessageType.REQUEST,
        priority: MessagePriority = MessagePriority.NORMAL,
        metadata: Dict[str, Any] = None
    ) -> A2AMessage:
        """Send an intelligent message to another agent"""
        
        try:
            # Generate intelligent content if this is a response
            if message_type in [MessageType.RESPONSE, MessageType.TASK_COMPLETION]:
                context = f"Responding to: {content}"
                intelligent_content = await self._get_intelligent_response(content, context)
            else:
                intelligent_content = content
            
            # Create the A2A message
            message = A2AMessage(
                id=self._generate_message_id(),
                conversation_id=self._get_conversation_id(to_agent),
                from_agent=self.name,
                from_role=self.role,
                to_agent=to_agent,
                to_role=to_role,
                message_type=message_type,
                content=intelligent_content,
                priority=priority,
                metadata=metadata or {}
            )
            
            # Log the message
            logger.info(f"📤 {self.name} → {to_agent}: {message.message_type.value}")
            
            # Update metrics
            self.performance_metrics["messages_sent"] += 1
            
            # Store in conversation history
            self.conversation_history.append({
                "direction": "sent",
                "message": message.dict(),
                "timestamp": datetime.now().isoformat()
            })
            
            # Display the message beautifully
            self._display_sent_message(message)
            
            return message
            
        except Exception as e:
            logger.error(f"❌ {self.name} failed to send message: {str(e)}")
            self.performance_metrics["failed_interactions"] += 1
            raise
    
    async def receive_message(self, message: A2AMessage) -> str:
        """Receive and process a message from another agent"""
        
        try:
            logger.info(f"📥 {self.name} received {message.message_type.value} from {message.from_agent}")
            
            # Update metrics
            self.performance_metrics["messages_received"] += 1
            
            # Store in conversation history
            self.conversation_history.append({
                "direction": "received",
                "message": message.dict(),
                "timestamp": datetime.now().isoformat()
            })
            
            # Display the received message
            self._display_received_message(message)
            
            # Generate intelligent response based on message content and type
            context = f"Message from {message.from_agent} ({message.from_role}): {message.content}"
            response_content = await self._get_intelligent_response(message.content, context)
            
            self.performance_metrics["successful_interactions"] += 1
            
            return response_content
            
        except Exception as e:
            logger.error(f"❌ {self.name} failed to process message: {str(e)}")
            self.performance_metrics["failed_interactions"] += 1
            return f"Error processing message: {str(e)}"
    
    def _display_sent_message(self, message: A2AMessage):
        """Display sent message in a beautiful format"""
        print(f"\n📤 {self.name} → {message.to_agent}")
        print(f"   🏷️  Type: {message.message_type.value}")
        print(f"   📝 Content: {message.content}")
        print(f"   🆔 ID: {message.id}")
        print(f"   ⚡ Priority: {message.priority.value}")
    
    def _display_received_message(self, message: A2AMessage):
        """Display received message in a beautiful format"""
        print(f"\n📥 {self.name} ← {message.from_agent}")
        print(f"   🏷️  Type: {message.message_type.value}")
        print(f"   📝 Content: {message.content}")
        print(f"   🆔 ID: {message.id}")
    
    def get_performance_summary(self) -> Dict[str, Any]:
        """Get agent performance metrics"""
        return {
            "agent_name": self.name,
            "agent_role": self.role,
            "specialization": self.specialization,
            "metrics": self.performance_metrics,
            "conversation_count": len(self.conversation_history),
            "uptime": "Active"
        }

print("🏗️ Intelligent A2A Agent Base Class created successfully!")
print("✅ Features included:")
print("   🧠 OpenAI integration for intelligent responses")
print("   📋 Structured A2A message protocol")
print("   📊 Performance monitoring and metrics")
print("   🔍 Comprehensive logging and traceability")
print("   ⚡ Error handling and fallback responses")

**Excellent!** Our intelligent agent base class is ready. Every agent built from this foundation will have:

- ✅ **OpenAI integration** - Intelligent, context-aware responses
- ✅ **Performance monitoring** - Response times and success rates
- ✅ **Message validation** - Structured, reliable communication
- ✅ **Error resilience** - Graceful failure handling
- ✅ **Complete traceability** - Full conversation history

---

## 🎯 Step 4: Create Your First Intelligent Agent Pair

Now for the exciting part! We'll create two specialized agents that will work together to handle customer support scenarios.

### Meet Your Intelligent Agents:

**🎯 CustomerServiceAgent** - First point of contact, routes inquiries intelligently  
**🔧 TechnicalSupportAgent** - Handles technical issues with specialized knowledge

In [ ]:
class CustomerServiceAgent(IntelligentA2AAgent):
    """Intelligent customer service agent with routing capabilities"""
    
    def __init__(self):
        super().__init__(
            name="CustomerServiceAgent",
            role="Customer Service Representative",
            specialization="Customer inquiry routing, basic support, and escalation management"
        )
        
        # Customer service specific capabilities
        self.supported_categories = [
            "general_inquiry",
            "technical_issue",
            "billing_question",
            "product_information",
            "complaint",
            "feature_request"
        ]
    
    async def analyze_customer_inquiry(self, inquiry: str) -> Dict[str, Any]:
        """Analyze customer inquiry and determine routing"""
        
        analysis_prompt = f"""
Analyze this customer inquiry and provide routing information:

Customer inquiry: "{inquiry}"

Please determine:
1. Category (general_inquiry, technical_issue, billing_question, product_information, complaint, feature_request)
2. Urgency level (low, normal, high, urgent)
3. Whether it needs technical support (true/false)
4. Brief summary of the issue

Respond in this exact format:
Category: [category]
Urgency: [urgency]
Needs Technical Support: [true/false]
Summary: [brief summary]
"""
        
        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": analysis_prompt}],
                max_tokens=150,
                temperature=0.3
            )
            
            analysis_text = response.choices[0].message.content
            
            # Parse the analysis (simple parsing for demo)
            lines = analysis_text.strip().split('\n')
            analysis = {}
            
            for line in lines:
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip().lower().replace(' ', '_')
                    value = value.strip()
                    analysis[key] = value
            
            return {
                "category": analysis.get("category", "general_inquiry"),
                "urgency": analysis.get("urgency", "normal"),
                "needs_technical_support": analysis.get("needs_technical_support", "false").lower() == "true",
                "summary": analysis.get("summary", "Customer inquiry analysis")
            }
            
        except Exception as e:
            logger.error(f"❌ Failed to analyze inquiry: {str(e)}")
            return {
                "category": "general_inquiry",
                "urgency": "normal",
                "needs_technical_support": False,
                "summary": "Unable to analyze inquiry"
            }

class TechnicalSupportAgent(IntelligentA2AAgent):
    """Intelligent technical support agent with specialized knowledge"""
    
    def __init__(self):
        super().__init__(
            name="TechnicalSupportAgent",
            role="Technical Support Specialist",
            specialization="Software troubleshooting, API issues, configuration problems, and technical guidance"
        )
        
        # Technical support specific capabilities
        self.expertise_areas = [
            "API Integration",
            "Authentication Issues",
            "Performance Problems",
            "Configuration Setup",
            "Error Troubleshooting",
            "Software Installation"
        ]
    
    async def provide_technical_solution(self, issue_description: str) -> Dict[str, Any]:
        """Provide technical solution for the given issue"""
        
        solution_prompt = f"""
As a technical support specialist, provide a solution for this issue:

Issue: "{issue_description}"

Please provide:
1. Root cause analysis
2. Step-by-step solution
3. Prevention tips
4. Estimated resolution time

Keep the response professional and actionable.
"""
        
        try:
            response = self.openai_client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": solution_prompt}],
                max_tokens=300,
                temperature=0.3
            )
            
            solution = response.choices[0].message.content
            
            return {
                "solution": solution,
                "resolved": True,
                "specialist": self.name
            }
            
        except Exception as e:
            logger.error(f"❌ Failed to generate technical solution: {str(e)}")
            return {
                "solution": "I'm experiencing technical difficulties. Please try again later or contact our support team directly.",
                "resolved": False,
                "specialist": self.name
            }

# Create the intelligent agent pair
print("🤖 Creating Intelligent A2A Agent Pair:")
print("=" * 45)

# Initialize both agents
customer_service = CustomerServiceAgent()
technical_support = TechnicalSupportAgent()

print("\n✅ Intelligent Agent Pair Created Successfully!")
print(f"🎯 {customer_service.name}: {customer_service.specialization}")
print(f"🔧 {technical_support.name}: {technical_support.specialization}")
print("\n🚀 Ready for intelligent A2A communication!")

**Amazing!** Your intelligent agent pair is ready. Each agent has:

- ✅ **Specialized knowledge** - Domain-specific capabilities
- ✅ **Intelligent analysis** - AI-powered decision making
- ✅ **Professional communication** - Context-aware responses
- ✅ **Error resilience** - Graceful handling of issues

---

## 🎭 Step 5: Test Intelligent Agent Communication

**The moment of truth!** Let's watch our intelligent agents collaborate to solve a real customer support scenario. This demonstrates the power of A2A protocol with AI intelligence.

In [ ]:
# Test Scenario: Customer has an API authentication issue
print("🎭 INTELLIGENT A2A COMMUNICATION TEST")
print("=" * 50)
print("📋 Scenario: Customer API Authentication Issue")
print("🎯 Testing: Intelligent routing and collaboration")
print()

# Simulate a customer inquiry
customer_inquiry = """
Hi, I'm having trouble with your API. I keep getting 401 Unauthorized errors 
when trying to authenticate. I've checked my API key multiple times and it 
looks correct. This is blocking our production deployment. Please help urgently!
"""

print(f"💬 Customer Inquiry:")
print(f"   {customer_inquiry.strip()}")
print()

# Step 1: Customer Service analyzes the inquiry
print("🔍 Step 1: CustomerServiceAgent analyzes inquiry...")
analysis = await customer_service.analyze_customer_inquiry(customer_inquiry)

print(f"📊 Analysis Results:")
print(f"   📂 Category: {analysis['category']}")
print(f"   🚨 Urgency: {analysis['urgency']}")
print(f"   🔧 Needs Technical Support: {analysis['needs_technical_support']}")
print(f"   📝 Summary: {analysis['summary']}")
print()

In [ ]:
# Step 2: Customer Service sends initial response and routes to Technical Support
print("📞 Step 2: CustomerServiceAgent provides initial response...")

# Customer Service generates intelligent response
initial_response = await customer_service.send_message(
    to_agent="Customer",
    to_role="Customer",
    content=f"Thank you for contacting support. I understand you're experiencing API authentication issues. Based on the urgency of this production-blocking issue, I'm immediately routing you to our Technical Support specialist who can resolve this quickly.",
    message_type=MessageType.RESPONSE,
    priority=MessagePriority.HIGH,
    metadata={
        "analysis": analysis,
        "customer_inquiry": customer_inquiry.strip()
    }
)

print()

In [ ]:
# Step 3: Customer Service hands off to Technical Support
print("🔄 Step 3: Intelligent handoff to TechnicalSupportAgent...")

handoff_message = await customer_service.send_message(
    to_agent=technical_support.name,
    to_role=technical_support.role,
    content=f"""URGENT: Production-blocking API authentication issue requiring immediate attention.
    
Customer Issue: {analysis['summary']}
Priority: {analysis['urgency'].upper()}
Category: {analysis['category']}

Customer Description: {customer_inquiry.strip()}

Please provide technical resolution ASAP.""",
    message_type=MessageType.HANDOFF,
    priority=MessagePriority.HIGH,
    metadata={
        "analysis": analysis,
        "original_inquiry": customer_inquiry.strip(),
        "urgency_reason": "Production deployment blocked"
    }
)

print()

In [ ]:
# Step 4: Technical Support receives and processes the handoff
print("🛠️  Step 4: TechnicalSupportAgent processes handoff...")

tech_response = await technical_support.receive_message(handoff_message)

print(f"\n🧠 TechnicalSupportAgent intelligent response:")
print(f"   {tech_response}")
print()

In [ ]:
# Step 5: Technical Support provides detailed solution
print("💡 Step 5: TechnicalSupportAgent provides technical solution...")

# Generate detailed technical solution
solution = await technical_support.provide_technical_solution(
    "API returns 401 Unauthorized errors despite correct API key, blocking production deployment"
)

# Send solution back to Customer Service for relay
solution_message = await technical_support.send_message(
    to_agent=customer_service.name,
    to_role=customer_service.role,
    content=f"""RESOLUTION PROVIDED for urgent API authentication issue:

{solution['solution']}

Status: {'RESOLVED' if solution['resolved'] else 'NEEDS FOLLOW-UP'}
Specialist: {solution['specialist']}

Please relay this solution to the customer immediately.""",
    message_type=MessageType.TASK_COMPLETION,
    priority=MessagePriority.HIGH,
    metadata={
        "solution_data": solution,
        "issue_resolved": solution['resolved'],
        "resolution_time": datetime.now().isoformat()
    }
)

print()

In [ ]:
# Step 6: Customer Service receives solution and confirms completion
print("✅ Step 6: CustomerServiceAgent receives solution and closes loop...")

final_response = await customer_service.receive_message(solution_message)

print(f"\n🎯 CustomerServiceAgent final response:")
print(f"   {final_response}")

print("\n" + "=" * 50)
print("🎉 INTELLIGENT A2A COMMUNICATION TEST COMPLETED!")
print("=" * 50)

---

## 📊 Step 6: Performance Analysis & Metrics

Let's analyze how well our intelligent agents performed during the communication test.

In [ ]:
# Generate performance reports for both agents
print("📊 INTELLIGENT A2A AGENT PERFORMANCE ANALYSIS")
print("=" * 55)

# Customer Service Agent Performance
cs_performance = customer_service.get_performance_summary()
print(f"🎯 {cs_performance['agent_name']} Performance:")
print(f"   📤 Messages Sent: {cs_performance['metrics']['messages_sent']}")
print(f"   📥 Messages Received: {cs_performance['metrics']['messages_received']}")
print(f"   ⚡ Avg Response Time: {cs_performance['metrics']['avg_response_time']:.2f}s")
print(f"   ✅ Successful Interactions: {cs_performance['metrics']['successful_interactions']}")
print(f"   ❌ Failed Interactions: {cs_performance['metrics']['failed_interactions']}")
print(f"   💬 Conversation History: {cs_performance['conversation_count']} entries")

print()

# Technical Support Agent Performance
ts_performance = technical_support.get_performance_summary()
print(f"🔧 {ts_performance['agent_name']} Performance:")
print(f"   📤 Messages Sent: {ts_performance['metrics']['messages_sent']}")
print(f"   📥 Messages Received: {ts_performance['metrics']['messages_received']}")
print(f"   ⚡ Avg Response Time: {ts_performance['metrics']['avg_response_time']:.2f}s")
print(f"   ✅ Successful Interactions: {ts_performance['metrics']['successful_interactions']}")
print(f"   ❌ Failed Interactions: {ts_performance['metrics']['failed_interactions']}")
print(f"   💬 Conversation History: {ts_performance['conversation_count']} entries")

print()

# Overall Network Performance
total_messages = (cs_performance['metrics']['messages_sent'] + 
                 cs_performance['metrics']['messages_received'] +
                 ts_performance['metrics']['messages_sent'] + 
                 ts_performance['metrics']['messages_received'])

total_successful = (cs_performance['metrics']['successful_interactions'] + 
                   ts_performance['metrics']['successful_interactions'])

total_failed = (cs_performance['metrics']['failed_interactions'] + 
               ts_performance['metrics']['failed_interactions'])

success_rate = (total_successful / (total_successful + total_failed)) * 100 if (total_successful + total_failed) > 0 else 100

print("🌐 Overall A2A Network Performance:")
print(f"   📊 Total Messages Exchanged: {total_messages}")
print(f"   ✅ Success Rate: {success_rate:.1f}%")
print(f"   🤖 Active Agents: 2")
print(f"   🔗 Protocol Version: A2A-v1.0")
print(f"   ⚡ Network Status: FULLY OPERATIONAL")

print("\n🎯 Key Achievements:")
print("   ✅ Intelligent inquiry analysis and routing")
print("   ✅ Context-aware inter-agent communication")
print("   ✅ Specialized technical problem resolution")
print("   ✅ Complete message traceability and monitoring")
print("   ✅ Error-resilient agent collaboration")

---

## 🧪 Step 7: Test Different Scenarios

Let's test our intelligent agents with different types of customer inquiries to see how they adapt their responses.

In [ ]:
# Test multiple scenarios to demonstrate agent intelligence
print("🧪 TESTING AGENT INTELLIGENCE WITH VARIED SCENARIOS")
print("=" * 55)

test_scenarios = [
    {
        "name": "Billing Question",
        "inquiry": "I was charged twice for my subscription this month. Can you help me get a refund?",
        "expected_category": "billing_question"
    },
    {
        "name": "Product Information",
        "inquiry": "What's the difference between your Pro and Enterprise plans? I need more details about API rate limits.",
        "expected_category": "product_information"
    },
    {
        "name": "Feature Request",
        "inquiry": "Would it be possible to add webhook support for real-time notifications? This would be very helpful for our integration.",
        "expected_category": "feature_request"
    }
]

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n🎭 Scenario {i}: {scenario['name']}")
    print(f"💬 Customer: {scenario['inquiry']}")
    
    # Analyze the inquiry
    analysis = await customer_service.analyze_customer_inquiry(scenario['inquiry'])
    
    print(f"🔍 Analysis:")
    print(f"   📂 Detected Category: {analysis['category']}")
    print(f"   🚨 Urgency Level: {analysis['urgency']}")
    print(f"   🔧 Technical Support Needed: {analysis['needs_technical_support']}")
    
    # Generate intelligent response
    response = await customer_service.send_message(
        to_agent="Customer",
        to_role="Customer",
        content=scenario['inquiry'],
        message_type=MessageType.RESPONSE,
        priority=MessagePriority.NORMAL if analysis['urgency'] == 'normal' else MessagePriority.HIGH
    )
    
    print(f"✨ Agent demonstrated intelligent categorization and appropriate response!")
    print("-" * 50)

print("\n🎉 All scenarios tested successfully!")
print("✅ Agents demonstrated intelligent adaptation to different inquiry types")
print("✅ Appropriate routing and priority assignment")
print("✅ Context-aware response generation")

---

## 🎉 Congratulations! You've Built Your First Intelligent A2A System

**This is a milestone moment!** You've successfully created a sophisticated Agent-to-Agent communication system that demonstrates the future of AI collaboration.

### 🏆 What You've Accomplished:

✅ **Intelligent Agent Communication** - Agents that think before responding using OpenAI  
✅ **Structured A2A Protocol** - Professional message format with full validation  
✅ **Business Problem Solving** - Real customer service automation in action  
✅ **Performance Monitoring** - Complete metrics and traceability  
✅ **Error Resilience** - Graceful handling of failures and edge cases  
✅ **Adaptive Intelligence** - Agents that adjust responses based on context  

### 🚀 The Magic You Just Witnessed:

- **CustomerServiceAgent** intelligently analyzed inquiries and routed them appropriately
- **TechnicalSupportAgent** provided specialized solutions using AI knowledge
- **Both agents collaborated** seamlessly using the A2A protocol
- **Complete automation** of a complex customer service workflow
- **Professional-grade implementation** ready for enterprise deployment

### 💡 Real-World Applications:

**This system can be immediately deployed for:**
- Customer support automation
- IT helpdesk ticketing
- Sales inquiry routing
- Technical documentation assistance
- Multi-department workflow coordination

### 🔮 What's Next?

In **Notebook 02: Agent Discovery and Registration**, you'll learn how to:
- Build agent networks that scale automatically
- Implement service discovery protocols
- Create agent registries for large networks
- Handle dynamic agent joining and leaving

### 📈 Your Learning Journey:

You've just completed **20% of the Master A2A course** and you already have:
- A working intelligent agent pair
- Understanding of A2A protocol fundamentals
- Real business application knowledge
- Foundation for building enterprise agent networks

---

## 💫 Share Your Success!

**You're now part of an exclusive group** - developers who can build intelligent agent networks using the A2A protocol. This is cutting-edge technology that most developers don't know exists yet.

**Share your achievement:**
- Post on LinkedIn about building your first A2A system
- Tag @pragatikunwer to connect with the A2A community
- Use hashtags: #AgenticAI #A2AProtocol #MultiAgent #AIAutomation

**Questions or want to show off your work?** Join the course discussion forum or connect on LinkedIn!

---

**Ready for the next level?** Open `02_Agent_Discovery_and_Registration.ipynb` and let's build agent networks that scale! 🌐🤖